In [1]:
#Gerekli kütüphanelerin bir kısmını import edelim

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings("ignore") #Temiz çıktı için

#Datasetlerimizi çağıralım

df_summary = pd.read_csv('Summary of Weather.csv')
df_stations = pd.read_csv('Weather Station Locations.csv')

#Kabaca gözlemleyelim
print("Summary Dataset:")
display(df_summary.head())
print("Stations Dataset:")
display(df_stations.head())

Summary Dataset:


,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,FB,FTI,ITH,PGT,TSHDSBRSGF,SD3,RHX,RHN,RVG,WTE
0,10001,1942-7-1,1.016,NaN,25.555556,22.222222,23.888889,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,1942-7-2,0,NaN,28.888889,21.666667,25.555556,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,1942-7-3,2.54,NaN,26.111111,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,1942-7-4,2.54,NaN,26.666667,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,1942-7-5,0,NaN,26.666667,21.666667,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Stations Dataset:


,WBAN,NAME,STATE/COUNTRY ID,LAT,LON,ELEV,Latitude,Longitude
0,33013,AIN EL,AL,3623N,00637E,611,36.383333,6.650000
1,33031,LA SENIA,AL,3537N,00037E,88,35.616667,0.583333
2,33023,MAISON BLANCHE,AL,3643N,00314E,23,36.716667,3.216667
3,33044,TELERGMA,AL,3607N,00621E,754,36.116667,6.416667
4,12001,TINDOUF,AL,2741N,00809W,443,27.683333,-8.083333


In [3]:
#Infolarını inceleyelim
df_summary.info()
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119040 entries, 0 to 119039
Data columns (total 31 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STA          119040 non-null  int64  
 1   Date         119040 non-null  object 
 2   Precip       119040 non-null  object 
 3   WindGustSpd  532 non-null     float64
 4   MaxTemp      119040 non-null  float64
 5   MinTemp      119040 non-null  float64
 6   MeanTemp     119040 non-null  float64
 7   Snowfall     117877 non-null  object 
 8   PoorWeather  34237 non-null   object 
 9   YR           119040 non-null  int64  
 10  MO           119040 non-null  int64  
 11  DA           119040 non-null  int64  
 12  PRCP         117108 non-null  object 
 13  DR           533 non-null     float64
 14  SPD          532 non-null     float64
 15  MAX          118566 non-null  float64
 16  MIN          118572 non-null  float64
 17  MEA          118542 non-null  float64
 18  SNF          117877 non-

In [4]:
#Yüksek oranda boş olan sütunları silelim
df_summary.drop(columns=["WTE","RVG","RHN","RHX","SD3","PGT","TSHDSBRSGF",
                         "ITH","FTI","FB","FT","SND","DR","SPD","PoorWeather","WindGustSpd"],
                         axis=1,
                         inplace=True)

In [5]:
#Benzer değerleri tutan sütunlari silelim
df_summary.drop(columns=["MAX","MIN","MEA","PRCP","SNF","YR","MO","DA"],axis=1,inplace=True)

In [6]:
#Tip dönüşümü ve yanlış değer düzenleme

#Not: 'T' değeri gözlem yapılamayacak kadar az yağış(0 mm - 0.1 mm) anlamına geldiği için
#0.05 gibi modeli yanıltmayacak bir değer ataması yaptık

df_summary["Precip"] = df_summary["Precip"].replace("T", 0.05)
df_summary["Precip"] = df_summary["Precip"].astype(float)

In [7]:
df_summary = df_summary[df_summary["Snowfall"] != "#VALUE!"]
df_summary["Snowfall"] = df_summary["Snowfall"].astype(float)
df_summary.dropna(subset=["Snowfall"],inplace=True)

In [8]:
df_summary["Date"] = pd.to_datetime(df_summary["Date"])
df_summary["Year"] = df_summary["Date"].dt.year
df_summary["Month"] = df_summary["Date"].dt.month

In [9]:
df_stations.drop(columns=['LAT', 'LON'], inplace=True)

In [10]:
#Güvenlik kontrolü istasyon sayıları eşit mi?
print("Summary istasyon sayısı:", df_summary['STA'].nunique())
print("Stations istasyon sayısı:", df_stations['WBAN'].nunique())

# Eşleşmeyen istasyon var mı kontrol edelim
unique_sta = set(df_summary['STA'].unique())
unique_wban = set(df_stations['WBAN'].unique())
print("Eşleşmeyen istasyon sayısı:", len(unique_sta - unique_wban))


Summary istasyon sayısı: 158
Stations istasyon sayısı: 161
Eşleşmeyen istasyon sayısı: 0


In [11]:
# Gereksiz sütunları atalım
df_stations_clean = df_stations[['WBAN', 'Latitude', 'Longitude', 'ELEV']]

# Sütun isimlerini eşitleyelim
df_stations_clean.rename(columns={'WBAN': 'STA'}, inplace=True)

# Merge işlemini yapalım
df_merged = df_summary.merge(df_stations_clean, on='STA', how='left')

# Kontrol edelim
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117833 entries, 0 to 117832
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   STA        117833 non-null  int64         
 1   Date       117833 non-null  datetime64[ns]
 2   Precip     117833 non-null  float64       
 3   MaxTemp    117833 non-null  float64       
 4   MinTemp    117833 non-null  float64       
 5   MeanTemp   117833 non-null  float64       
 6   Snowfall   117833 non-null  float64       
 7   Year       117833 non-null  int32         
 8   Month      117833 non-null  int32         
 9   Latitude   117833 non-null  float64       
 10  Longitude  117833 non-null  float64       
 11  ELEV       117833 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int32(2), int64(2)
memory usage: 9.9 MB


In [46]:
#Bağımlı ve Bağımsız değişkenleri oluşturma

X = df_merged[["Precip","Snowfall","Latitude","Longitude","ELEV","Month"]]
y = df_merged[["MeanTemp"]]

#Train-Test-Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#PolynomialFeatures

from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=4)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

#Regresyon Modeli

from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train_poly, y_train)

#Tahmin
y_pred = model.predict(X_test_poly)

from sklearn.metrics import r2_score, mean_squared_error

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("R2:", r2)
print("RMSE:", rmse)


R2: 0.80958709089117
RMSE: 3.462276172318775


In [ ]:
#Aşırı öğrenme kontrolü
y_train_pred = model.predict(X_train_poly)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_pred)

print("R² (train):", round(r2_train, 3))
print("R² (test):", round(r2_test, 3))
print("Fark:", round(r2_train - r2_test, 3))

R² (train): 0.758
R² (test): 0.773
Fark: -0.015


In [ ]:
#Örnek girdileri tanımlayalım
samples = pd.DataFrame([
    # İstanbul kış (hafif yağış, düşük rakım)
    {"Precip": 5.0,  "Snowfall": 0.0, "Latitude": 41.0, "Longitude": 29.0, "ELEV": 40,  "Month": 1},
    # İzmir yaz (kuru, deniz seviyesi)
    {"Precip": 0.0,  "Snowfall": 0.0, "Latitude": 38.4, "Longitude": 27.1, "ELEV": 5,   "Month": 7},
    # Bursa sonbahar (yağışlı, biraz iç kesim)
    {"Precip": 10.0, "Snowfall": 0.0, "Latitude": 40.2, "Longitude": 29.0, "ELEV": 100, "Month": 11},
    # Erzurum kış (kuru/kar yok varsayımı ama yüksek rakım)
    {"Precip": 0.0,  "Snowfall": 0.0, "Latitude": 39.9, "Longitude": 41.3, "ELEV": 1900,"Month": 1},
])

#PolynomialFeatures ile aynı dönüşümü uygulayalım (poly degree=4, eğitimdeki nesne)
X_new_poly = poly.transform(samples)

# 3) Tahmin yapma
y_new_pred = model.predict(X_new_poly).ravel()

# 4) Sonucu tabloya ekleme ve yuvarlama
out = samples.copy()
out["Predicted_MeanTemp"] = np.round(y_new_pred, 2)
print(out)


   Precip  Snowfall  Latitude  Longitude  ELEV  Month  Predicted_MeanTemp
0     5.0       0.0      41.0       29.0    40      1                5.44
1     0.0       0.0      38.4       27.1     5      7               24.34
2    10.0       0.0      40.2       29.0   100     11               14.76
3     0.0       0.0      39.9       41.3  1900      1              -16.43


### Model Performansı ve Yorum (Polynomial Regression, degree=4)

Test setinde **R² = 0.8096**, **RMSE = 3.46°C** elde edilmiştir. 
Bu sonuç, modelin ortalama sıcaklıktaki değişimin yaklaşık **%81’ini** açıkladığını ve tahmin hatasının tipik olarak **±3–4°C** bandında olduğunu gösterir.

Model; enlem, boylam ve rakımın fiziksel etkilerini, ay ve yağışın doğrusal olmayan etkilerini yakalayabilmektedir. 
Önceki denemelerde train ve test R² değerlerinin birbirine yakın çıkması, **overfitting olmadığını** ve modelin **iyi genelleştiğini** destekler.

Kısa değerlendirme:
- **Güçlü yönler:** Yüksek açıklama gücü, fiziksel olarak tutarlı ilişkiler (rakım ↑ → sıcaklık ↓; yaz/kış döngüsü).
- **Sınırlamalar:** Yüksek dereceli polinomlar veri aralığı dışına taşınca (extrapolation) hatayı büyütebilir.
